In [7]:
SEED = 42
from random import seed as py_seed
py_seed(SEED)
from numpy.random import seed as np_seed
np_seed(SEED)
from tensorflow import random as tf_random
tf_random.set_seed(SEED)

In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD, RMSprop
from keras.utils import np_utils
from keras.models import load_model
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.nasnet import NASNetMobile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import random
from sklearn.model_selection import KFold, train_test_split
import pickle


In [9]:
# Data comes from here. This should contain a bunch of folders of classes, each containing examples
#DATA_PATH = Path('food-101-original-source/images')
DATA_PATH = Path('input')
SAVE_PATH = '25_runs/'

# Get class (folder) names
classes = sorted([entry.name for entry in DATA_PATH.iterdir() if entry.is_dir()])

SPLIT = 25
random.shuffle(classes) # shuffle the classes 
classes = classes[:SPLIT]
# Load list of filenames and classes, and convert to numpy array to allow array slicing
# For k-fold validation
dataset = np.array([
    (f"{clazz}/{entry.name}", clazz)
    for clazz in classes
    for entry in (DATA_PATH / clazz).iterdir()
])

In [11]:
print(classes)

['breakfast_burrito', 'beef_carpaccio', 'bruschetta', 'falafel', 'club_sandwich', 'clam_chowder', 'deviled_eggs', 'caesar_salad', 'beef_tartare', 'dumplings', 'chocolate_cake', 'croque_madame', 'churros', 'chicken_curry', 'cannoli', 'eggs_benedict', 'escargots', 'crab_cakes', 'caprese_salad', 'chocolate_mousse', 'chicken_wings', 'donuts', 'filet_mignon', 'cup_cakes', 'chicken_quesadilla']


In [12]:
# create data generators

# Rescale 0-255 to 0-1
RESCALE=1./255

# This type of data generator is used to train the model
train_datagen = ImageDataGenerator(
    rescale=RESCALE,
    # Factors used to control
    rotation_range=45,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.125,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.125,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images

)

# This type of generator is used to test the model
test_datagen = ImageDataGenerator(
    rescale=RESCALE,
)

In [13]:
def create_model(input_shape, num_classes):
    # Base model, with weights pre-trained on ImageNet.
    base_model = NASNetMobile(input_shape, weights='imagenet', include_top=False)

    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    optimizer = RMSprop()

    model.compile(
        loss = 'categorical_crossentropy',
        optimizer = optimizer,
        metrics = ['accuracy']
    )
    
    return model

In [14]:
# Image data generator settings
TARGET_SIZE = (224,224)
COLOR_MODE = "rgb"
BATCH_SIZE = 64
CLASS_MODE = "categorical"


# Training settings
NUM_FOLDS = 5
VALIDATION_SPLIT = 0.1
NUM_EPOCHS = 5

# Model settings
kf = KFold(n_splits=NUM_FOLDS, random_state=SEED, shuffle=True)
results = []
cur_fold = 1

# saving classes used by this model
with open(SAVE_PATH+'classes_25.txt', 'w') as f:
    f.writelines("%s\n" % clss for clss in classes)


for train_index, test_index in kf.split(dataset):
    # Print a message
    print(f"Fold {cur_fold} of {NUM_FOLDS}")
    cur_fold += 1
    
    # Extract train dataset
    d_train = dataset[train_index]
    # Split train dataset to form train and validation dataset
    d_train, d_val = train_test_split(dataset, test_size = VALIDATION_SPLIT)
    
    # These are used to convert the filename,class pairs to dataframes
    FILENAME_COL = "filename"
    CLASS_COL = "class"
    COLUMNS = [FILENAME_COL, CLASS_COL]
    # Convert to dataframes
    d_train = pd.DataFrame(d_train, columns = COLUMNS)
    d_val = pd.DataFrame(d_val, columns = COLUMNS)
    
    # Create training data generators
    # Training image data generator
    train_it = train_datagen.flow_from_dataframe(
        d_train,
        directory = DATA_PATH,
        x_col = FILENAME_COL,
        y_col = CLASS_COL,
        target_size = TARGET_SIZE,
        color_mode = COLOR_MODE,
        batch_size = BATCH_SIZE,
        class_mode = CLASS_MODE,
        shuffle = True,
        seed = SEED
    )
    # Validation image data generator
    val_it = test_datagen.flow_from_dataframe(
        d_val,
        directory = DATA_PATH,
        x_col = FILENAME_COL,
        y_col = CLASS_COL,
        target_size = TARGET_SIZE,
        color_mode = COLOR_MODE,
        batch_size = BATCH_SIZE,
        class_mode = CLASS_MODE,
        shuffle = False,
    )
    
    # Perform training
    # Build the model
    input_shape = (*TARGET_SIZE,3)
    num_classes = len(classes)
    model = create_model(input_shape, num_classes)
    
    # Train the model
    history = model.fit(
        train_it,
        validation_data=val_it,
        epochs=NUM_EPOCHS,
        workers=6,
        max_queue_size=100,
        verbose=True
    )
    
    # Extract test dataset and convert to dataframe
    d_test = pd.DataFrame(dataset[test_index], columns = COLUMNS)
    # Create test image data generator
    test_it = test_datagen.flow_from_dataframe(
        d_test,
        directory = DATA_PATH,
        x_col = FILENAME_COL,
        y_col = CLASS_COL,
        target_size = TARGET_SIZE,
        color_mode = COLOR_MODE,
        batch_size = BATCH_SIZE,
        class_mode = CLASS_MODE,
        shuffle = False,
    )
    
    # Save history to a file 
    with open(SAVE_PATH+"history_25_"+str(cur_fold-1)+".pkl", 'wb') as file:
        pickle.dump(history, file)   
        
    # saving labels to a file (y_test)
    with open(SAVE_PATH+"y_test_25_"+str(cur_fold-1)+".pkl", 'wb') as file:
        pickle.dump(test_it.classes, file)

    # predict model
    y_predict = model.predict(test_it)
    
    # save it to a file (Y_predict)
    with open(SAVE_PATH+"y_predict_25_"+str(cur_fold-1)+".pkl", 'wb') as file:
        pickle.dump(y_predict, file)
    
    # Test
    fold_results = model.evaluate(test_it)
    
    results.append(fold_results)
    print(f"Results for current fold: {fold_results}")

Fold 1 of 5
Found 22500 validated image filenames belonging to 25 classes.
Found 2500 validated image filenames belonging to 25 classes.
Epoch 1/5
352/352 [==============================] - 1149s 3s/step - loss: 2.4324 - accuracy: 0.3070 - val_loss: 1.0851 - val_accuracy: 0.4316
Epoch 2/5
352/352 [==============================] - 1137s 3s/step - loss: 1.9213 - accuracy: 0.4448 - val_loss: 1.0562 - val_accuracy: 0.4816
Epoch 3/5
352/352 [==============================] - 1126s 3s/step - loss: 1.7993 - accuracy: 0.4790 - val_loss: 1.0392 - val_accuracy: 0.4884
Epoch 4/5
352/352 [==============================] - 1182s 3s/step - loss: 1.7750 - accuracy: 0.4816 - val_loss: 0.9737 - val_accuracy: 0.4972
Epoch 5/5
352/352 [==============================] - 1170s 3s/step - loss: 1.7371 - accuracy: 0.4952 - val_loss: 1.0224 - val_accuracy: 0.5160
Found 5000 validated image filenames belonging to 25 classes.
79/79 [==============================] - 208s 3s/step
Results for current fold: [2.053